# Armoria API

In [26]:
import requests
import json
from PIL import Image
from io import BytesIO

class ArmoriaAPIWrapper:
    def __init__(self, size, format, coa):
        self.format = format
        payload = {"size": size, 
                   "format": format,
                   "coa": json.dumps(coa)
                  }
        self.r = requests.get('https://armoria.herokuapp.com/', params=payload)
#         print(self.r.url)
    
    def get_image_bytes(self):
        i = Image.open(BytesIO(self.r.content))
        return i
    
    def show_image(self):
        i = self.get_image_bytes()
        i.show()
    
    def save_image(self, full_path):
        i = self.get_image_bytes()    
        i.save(full_path, 'png')



In [17]:
coa_lion = {"t1":"or",
       "shield":"heater",
       "charges":[{"charge":"lionRampant",
                   "t":"gules",
                   "p":"e",
                   "size":1.5}]
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_lion)
api.show_image()

coa_cross = {"t1":"argent",
       "shield":"heater",
       "charges":[{"charge":"crossMoline",
                   "t":"azure",
                   "p":"e",
                   "size":1.5}]
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_cross)
api.show_image()


https://armoria.herokuapp.com/?size=500&format=png&coa=%7B%22t1%22%3A+%22or%22%2C+%22shield%22%3A+%22heater%22%2C+%22charges%22%3A+%5B%7B%22charge%22%3A+%22lionRampant%22%2C+%22t%22%3A+%22gules%22%2C+%22p%22%3A+%22e%22%2C+%22size%22%3A+1.5%7D%5D%7D
https://armoria.herokuapp.com/?size=500&format=png&coa=%7B%22t1%22%3A+%22argent%22%2C+%22shield%22%3A+%22heater%22%2C+%22charges%22%3A+%5B%7B%22charge%22%3A+%22crossMoline%22%2C+%22t%22%3A+%22azure%22%2C+%22p%22%3A+%22e%22%2C+%22size%22%3A+1.5%7D%5D%7D


# Map our dataset to the API keywords

In [23]:
LION_MODIFIERS_MAP = {
    'lion rampant': 'lionRampant',
    'lion passt': 'lionPassant',
    'lion passt guard': 'lionPassantGuardant',
    "lion's head": 'lionHeadCaboshed'
}

CROSS_MODIFIERS_MAP = {
    'cross': 'crossHummetty' ,
    'cross moline': 'crossMoline',
    'cross patonce': 'crossPatonce',
}


MODIFIERS_MAP = {**LION_MODIFIERS_MAP, **CROSS_MODIFIERS_MAP} 

COLORS_MAP = { 'A': 'argent', # silver
              'B': 'azure', # blue
              'O': 'or', # gold 
              'S': 'sable', # black
              'G': 'gules', # red
              'V': 'vert' # green
             }
#               'E':'',
#               'X': '', 
#               'Z': ''}



# Generate Permutations

In [24]:
import itertools
import src.alphabet as alphabet

all_possible_permutations = []

# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product( list(COLORS_MAP.keys()), list( LION_MODIFIERS_MAP.keys()))))

# cross, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product( list(COLORS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()))))

all_possible_permutations = permutations1 + permutations2

print('Total number of permutations:', len(all_possible_permutations))
all_possible_permutations

Total number of permutations: 42


[('A', 'lion rampant'),
 ('A', 'lion passt'),
 ('A', 'lion passt guard'),
 ('A', "lion's head"),
 ('B', 'lion rampant'),
 ('B', 'lion passt'),
 ('B', 'lion passt guard'),
 ('B', "lion's head"),
 ('O', 'lion rampant'),
 ('O', 'lion passt'),
 ('O', 'lion passt guard'),
 ('O', "lion's head"),
 ('S', 'lion rampant'),
 ('S', 'lion passt'),
 ('S', 'lion passt guard'),
 ('S', "lion's head"),
 ('G', 'lion rampant'),
 ('G', 'lion passt'),
 ('G', 'lion passt guard'),
 ('G', "lion's head"),
 ('V', 'lion rampant'),
 ('V', 'lion passt'),
 ('V', 'lion passt guard'),
 ('V', "lion's head"),
 ('A', 'cross'),
 ('A', 'cross moline'),
 ('A', 'cross patonce'),
 ('B', 'cross'),
 ('B', 'cross moline'),
 ('B', 'cross patonce'),
 ('O', 'cross'),
 ('O', 'cross moline'),
 ('O', 'cross patonce'),
 ('S', 'cross'),
 ('S', 'cross moline'),
 ('S', 'cross patonce'),
 ('G', 'cross'),
 ('G', 'cross moline'),
 ('G', 'cross patonce'),
 ('V', 'cross'),
 ('V', 'cross moline'),
 ('V', 'cross patonce')]

In [27]:
def get_armoria_api_payload(label):
    color = label[0]
    charge = label[1]
    api_color = COLORS_MAP[color]
    api_charge = MODIFIERS_MAP[charge]

    coa = {"t1":"or",
       "shield":"heater",
       "charges":[{"charge":api_charge,
                   "t": api_color,
                   "p":"e",
                   "size":1.5}]
      }

    return coa

FOLDER_NAME = 'generated-data-api'
counter = 1
for label in all_possible_permutations:
    
    payload = get_armoria_api_payload(label)
    
    api = ArmoriaAPIWrapper(size = 500,format="png", coa=payload)
    
    image_full_path = FOLDER_NAME + '/' + 'image_' + str(counter) + '.png'
    
    api.save_image(image_full_path)
    
    print('Image "{}" for label "{}" has been generated succfully' .format(image_full_path, label) )
    
    counter = counter + 1


Image "generated-data-api/image_1.png" for label "('A', 'lion rampant')" has been generated succfully
Image "generated-data-api/image_2.png" for label "('A', 'lion passt')" has been generated succfully
Image "generated-data-api/image_3.png" for label "('A', 'lion passt guard')" has been generated succfully
Image "generated-data-api/image_4.png" for label "('A', "lion's head")" has been generated succfully
Image "generated-data-api/image_5.png" for label "('B', 'lion rampant')" has been generated succfully
Image "generated-data-api/image_6.png" for label "('B', 'lion passt')" has been generated succfully
Image "generated-data-api/image_7.png" for label "('B', 'lion passt guard')" has been generated succfully
Image "generated-data-api/image_8.png" for label "('B', "lion's head")" has been generated succfully
Image "generated-data-api/image_9.png" for label "('O', 'lion rampant')" has been generated succfully
Image "generated-data-api/image_10.png" for label "('O', 'lion passt')" has been